# Input Selection
Select `index` value from 0 to 14

In [7]:
# Assign the index manually if needed
index = int(input("Enter an index value (0-14): "))

In [2]:
# Required for PydanticAI to work with Jupyter (nested event loops)
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pastel.classifiers import classify_insurance_image
from pastel.parsers import parse_assertion, parse_evidence, check_grammar
from pastel.evaluation import consolidate_evaluations
from IPython.display import display, Markdown

import pandas as pd
import logfire
import os
from pastel.helpers import (
    load_images_from_directory,
    create_consolidated_image,
    export_evaluation_to_markdown,
)
from pastel.models import (
    InsightPlots,
    InputModel,
)

if os.getenv("PYDANTIC_LOGFIRE_TOKEN"):
    logfire.configure(token=os.getenv("PYDANTIC_LOGFIRE_TOKEN"))
    logfire.instrument_openai()
    logfire.instrument_anthropic()

# Insight Validation
- Load datasets
- Collect images
- Parse Insight
  - conclusion
  - supporting premises
- Evaluate premises
- Check grammar

In [ ]:
insights_df = pd.read_excel("../data/Insights.xlsx")

input_model = InputModel(
    name=insights_df["programname"][index],
    insight=insights_df["insight"][index],
    line_of_business=insights_df["line_of_business"][index],
)

assertion = await parse_assertion(input_model)
insight = await parse_evidence(assertion)

# Retrieve data sets
lrs = pd.read_excel("../data/lrs.xlsx")
lrs_data = lrs.loc[lrs["programname"] == insight.name].drop(columns=["programname"])

images = InsightPlots(plots=load_images_from_directory(f"../data/{insight.name.replace('/', '-')}"))
consolidated_image = create_consolidated_image(lrs_data, images, insight)
premises = await classify_insurance_image(consolidated_image, insight.evidence)
grammar = await check_grammar(insight)
final_evaluation = await consolidate_evaluations(insight, premises, grammar)

# Show Validation Resuls
- Display report
- Write markdown to `../reports/` folder
---

In [ ]:
display(Markdown(f"---"))
display(Markdown("# Overall Assessment\n\n---"))
display(Markdown(f'#### "{insight.insight}"<br>'))
display(Markdown(f"## {final_evaluation.overall_valid}"))
display(Markdown(final_evaluation.reasoning))

display(Markdown("---"))
display(Markdown("## Conclusion\n\n"))
display(Markdown(f'#### "{insight.conclusion}"<br><br>'))

display(Markdown(f"---"))
display(Markdown("## Premises\n\n"))

if not premises:
    display(Markdown("####No premises found"))

else:
    for i, premise in enumerate(premises):
        display(Markdown(f"#### {i+1}. {premise.claim}"))
        display(Markdown(f"**Status** <br>{premise.status} <br>{premise.confidence} confidence"))
        display(Markdown(f"**Rationale** <br>{premise.reasoning}<br><br>"))

display(Markdown(f"---"))
if grammar.errors:
    error_list = "\n".join([f"- {error}" for error in grammar.errors])
    display(Markdown("## Grammar\n\n" + error_list))
else:
    display(Markdown("## Grammar\n\n"))
    display(Markdown("#### No grammatical errors found"))

In [ ]:

md_result = export_evaluation_to_markdown(insight, final_evaluation, premises, grammar)
display(Markdown(f"**{md_result}**"))